# Example of GITIII usage in the Alzheimer's disease (AD) example

In this tutorial, we demonstrate the functionalities of the cell-cell interaction (CCI) analyzing tool GITIII in:
1. Estimating the influence tensor that describe how each cell is influenced by its top k (default 50) nearest neighbors
2. Visualizing the relationship between the strength of interaction with the distance between two cells
3. Visualizing the functions of CCI pairs using UMAP: the pair of one sender cell influencing one receiver cell is called a CCI pair, and the estimated influence from the sender cell to the receiver cell (how much the gene expression in the receiver cell would change because of the existing of the sender cell) are treated as the features of the CCI pair. This function aim to visualize how different CCI pairs belonging to different cell type combinations differ from each other in terms of their functions
4. Prediction visualization: visualize the predicted cell expression v.s. predicted expression, can be state expression (have negative values) or raw expression (>0)
5. Information flow: Where are the strongest CCI pairs in the slide (tissue section), with arrows in the plot indicating the interaction from one sender cell to one receiver cell
6. Cell subtyping analysis: We first construct interpretable CCI-informed features for each cell, (how each cell type influence each measured genes in this cell), use these features to do Leiden clustering and UMAP visualization. Then differential expressed gene (DEG) analysis can be performed on these subtypes (subgroups), and we can also visualize how this cell ('s one target gene) is influenced by other cell types via heatmap.
7. Network analysis: Using partial linear regression to make statistical test of whether one cell type significantly influence one gene in the receiver cell type, forming a significant CCI network targeting each gene.

# Data downloading

The data used in this section is the SEA-AD dataset, which contains data from 366,272 cells across 69 images of the human middle temporal gyrus from 27 donors, obtained using MERFISH. The dataset includes the measurements of 140 genes, and we utilized the 24 "subclass" cell type annotation from the original study. For demonstration, we used the sample 'H20.33.001.CX28.MTG.02.007.1.02.03' (Figure 3a, 3b) to illustrate within-sample heterogeneity.

A processed dataset is available at https://drive.google.com/file/d/1KYpkDZHOlVJXZzKrfMbyyOl42syylw90/view?usp=sharing

Suppose the data is downloaded at the working directory with name "./AD.csv"

If you want to use your own data, the dataframe you input should have the following columns:

- genes (more than one column), as described below, these columns form the expression matrix. Values in these columns must be int or float
- "centerx": x coordinates of the cells. int or float
- "centery": y coordinates of the cells. int or float
- "section": which tissue slide this cell belongs to, since a dataset may contain more than one slide. string
- "subclass": the cell type annotation for this cell. string

You also need a list of measured genes in the dataset, like the code block below:

In [ ]:
genes=['PRRT4', 'GRIP2', 'FOXP2', 'PDZD2', 'KIAA1217', 'PALMD', 'LRRC4C', 'ASIC2', 'NPAS3', 'LUZP2', 'GRIN2A', 'NLGN1', 'NTNG2', 'TACR1', 'PDGFD', 'ANK1', 'DLX1', 'CBLN2', 'ZNF804A', 'CACNA2D3', 'CDH6', 'CD22', 'ETNPPL', 'CALB1', 'TSHZ2', 'FGF13', 'KIRREL3', 'ROBO1', 'RBFOX3', 'ASTN2', 'ID3', 'TH', 'TENM2', 'ZMAT4', 'CLSTN2', 'SEMA6D', 'HPSE2', 'BTBD11', 'LRP1B', 'NOS1', 'GPC5', 'SNTB1', 'COL11A1', 'TMEM255A', 'SATB2', 'SORCS3', 'FBXL7', 'GRM8', 'GALNTL6', 'NOSTRIN', 'DCC', 'SOX6', 'MEIS2', 'STXBP6', 'SMYD1', 'SCUBE1', 'LAMA4', 'CNTN5', 'GRM7', 'KCNMB2', 'CUX2', 'LAMP5', 'SLIT3', 'TAFA1', 'PRKG1', 'CSMD1', 'CNTNAP5', 'NFIA', 'FRMPD4', 'GRID2', 'HS6ST3', 'SORCS1', 'ATRNL1', 'ADAMTS3', 'SLC24A2', 'RBFOX1', 'TMEM132D', 'NKAIN2', 'PEX5L', 'TNR', 'DGKG', 'RFX3', 'UNC5B', 'HTR2A', 'RGS12', 'CACHD1', 'RORB', 'LRRK1', 'THEMIS', 'CARTPT', 'SLC32A1', 'GAD2', 'MOG', 'DCN', 'TOX', 'ZNF385D', 'PDE4B', 'GRIP1', 'ITGB8', 'PLD5', 'NPY', 'NDNF', 'SEMA3E', 'KAZN', 'DLC1', 'PLCB1', 'HCN1', 'ITGA8', 'EBF1', 'PRRX1', 'SLC14A1', 'EGFR', 'FEZF2', 'PAX6', 'ROBO2', 'SV2C', 'DCLK1', 'EYA4', 'RYR3', 'L3MBTL4', 'GRIN3A', 'CD74', 'RGS6', 'CTSS', 'KCNIP4', 'DACH1', 'HTR2C', 'PVALB', 'HS3ST2', 'GRIK3', 'FGF12', 'LHX6', 'VIP', 'CA10', 'ADAMTSL1', 'CHODL', 'SULF1', 'NRG1', 'NXPH2', 'TLL1']

# Initialize the influence tensor estimator

In [ ]:
# Import necessary python packages
import gitiii

estimator=gitiii.estimator.GITIII_estimator(df_path="./AD.csv",genes=genes,use_log_normalize=True,species="human",use_nichenetv2=True,visualize_when_preprocessing=False,distance_threshold=80,process_num_neighbors=50,num_neighbors=50,batch_size_train=256,lr=1e-4,epochs=50,node_dim=256,edge_dim=48,att_dim=8,batch_size_val=256,use_cell_type_embedding=True)

## Introduction of the parameters used in the code block above:

**Except for `df_path`, `genes`, `use_log_normalize`, and `species`, you can feel good to use all other hyperparameters by default as shown above, so generally speaking, you only need to fill in 4 parameters and prepare your own dataset and all other steps will be done automatically**

### Data you must input:

:param `df_path`: str, the path of your dataset, which should be a .csv file with columns of:
- genes (more than one column), as described below, these columns form the expression matrix.
            values in these columns must be int or float
- "centerx": x coordinates of the cells. int or float
- "centery": y coordinates of the cells. int or float
- "section": which tissue slide this cell belongs to, since a dataset may contain more than one slide. string
- "subclass": the cell type annotation for this cell. string

:param `genes`: list of str, a python list of measured gene names in the dataset

:param `use_log_normalize`: bool, whether to perform log-normalization log2(x+1) here for the expression matrix

- Attention: If you would like to use your own way of data normalization or have already normalized your expression matrix in the dataframe, choose False

:param `species`: str, what is the species of your dataset, must be one of "human" or "mouse"

---

### Hyperparameter settings (you can use the default one):

:param `use_nichenetv2`: bool, whether or not to include the ligands from nichenetv2, if not, only ligand-receptor pair from cellchat and neuronchat will be used

:param `visualize_when_preprocessing`: bool, whether to visualize the ST dataset when preprocessing

:param `distance_threshold`: float or int, if the distance between one cell and its nearest neighbor isabove this threshold, then we think this cell is moved during the preparation of the tissue slide in the wet lab experiment, we would not include this cell in the analysis

:param `process_num_neighbors`: int, how many k-nearest neighbor are needed to be calculated in preprocessing

:param `num_neighbors`: int, number of neighboring cells used to predict the cell state of the center cell

:param `batch_size_train`: int, batch size at the training step

:param `lr`: learning rate, float

:param `epochs`: int, number of training rounds

:param `node_dim`: int, embedding dimension for node in the graph transformer

:param `edge_dim`: int, embedding dimension for edge in the graph transformer

:param `att_dim`: int, dimension needed to calculate for one-head attention

:param `batch_size_val`: int, batch size at the evaluating step

# Preprocess the dataset

This includes calculating the top 50 nearest neighbor for each cell, splitting the cell type expression and cell state expression, this step only use CPU and takes about 15 minutes. 

For the next 3 step, I recommend to use script instead of using interactive software like jupyter notebook since it would took hours to finish.

In [ ]:
estimator.preprocess_dataset()

# Train the deep learning model

This step require GPU, the batch_size and model hyperparameter in the above default estimator hyperparameters roughly require 5G GPU memory, if you want it to train faster, you can increase the batch_size.

The training takes about 1.5 hours in this dataset (366,272 cells, 140 genes) with A5000 24G. The model converges at about 20-th epoch, and the software will automatically save the best model with the lowest mean square error on the validation set.

For your own dataset, the computation time would increase linearly with the number of cells in the dataset and sqrt(number of measured genes in the dataset).

In [ ]:
estimator.train()

# Calculate the influence tensor

This step require GPU, it would take about 2-10 minutes, be sure to have a large CPU memory (~100G, not GPU memory) if you have tissue sections that have many cells (>80000)

In [ ]:
estimator.calculate_influence_tensor()

# Visualize the spatial patterns

Here, we choose the tissue section (sample) 'H20.33.001.CX28.MTG.02.007.1.02.03' for demonstration

For analysis that you want to target a specific gene, we use "RORB" for demonstration

**Please pay attention that only gene that are measured in the dataset can be used, otherwise there would be error**

In [ ]:
sample='H20.33.001.CX28.MTG.02.007.1.02.03'

spatial_visualizer=gitiii.spatial_visualizer.Spatial_visualizer(sample=sample)

## Visualize the relationship between \[the averaged interaction strength between two cells\] with \[the distance between two cells\]

Estimate the distance scaler and visualize it, x-axis is distance or the rank of nearest neighbor,
        the y-axis can be proportional influence or the abs value of influence
        
:param `rank_or_distance`: x axis is distance or the order (rank) of nearest neighbor

:param `proportion_or_abs`: use the proportional influence (for each cell's each gene, the influence are all
            positive values that sum up to 1) or the abs value of interaction
            
:param `target_gene`: if None, calculate the distance scaler averaged over all target genes

:param `bins`, frac: parameters used of calculating and plot losses.

For example:

### The averaged abs value of interaction strength v.s. distance between two cells

In [ ]:
spatial_visualizer.plot_distance_scaler(rank_or_distance="distance",proportion_or_abs="abs",target_gene=None,bins=300, frac=0.003)

### The averaged abs value of interaction strength v.s. the order of nearest neighbor targeting gene "RORB"

In [ ]:
spatial_visualizer.plot_distance_scaler(rank_or_distance="rank",proportion_or_abs="proportion",target_gene="RORB",bins=300, frac=0.003)

## UMAP visualization of function of CCI pairs

Visualizing the functions of CCI pairs using UMAP: the pair of one sender cell influencing one receiver cell is called a CCI pair, and the estimated influence from the sender cell to the receiver cell (how much the gene expression in the receiver cell would change because of the existing of the sender cell) are treated as the features of the CCI pair. This function aim to visualize how different CCI pairs belonging to different cell type combinations differ from each other in terms of their functions

:param `select_topk`: For each receiver cell, how many strongest CCI pair should be selected for visualize
            one point on the UMAP is one CCI pair
            
:param `num_type_pair`: How many most frequent CCI type pair combination to show, since we can not demonstrate all cell_type_number*cell_type_number cell type combinations, there are too many colors

In [ ]:
spatial_visualizer.visualize_CCI_function(select_topk=5,num_type_pair=10)

## Prediction visualization

Visualize the predicted cell expression v.s. predicted expression, can be state expression (have negative values) or raw expression (>0)

In [ ]:
# Plot predicted expression (predicted cell state expression plus known cell type expression) v.s. real expression
spatial_visualizer.visualize_prediction(target_gene="RORB",plot_state=False)

In [ ]:
# Plot predicted cell state expression v.s. real cell state expression
spatial_visualizer.visualize_prediction(target_gene="RORB",plot_state=True)

## Information flow visualization
We want to find where are the strongest CCI pairs in the slide (tissue section), with arrows in the plot indicating the interaction from one sender cell to one receiver cell

:param target_gene: the information flow with respect to which gene you would like to visualized

:param select_topk: similar as before

:param use_neuron_layer: In the plot, whether to generalize the cell types to only excitatory neurons at
            different layers and Not_excitatory_neuron, by doing this, the plot may just look more layer-organized.
            
:param cutoff: In percentage, how many top CCI pairs out of select_topk*number_of_cell pairs to visualize.

In [ ]:
spatial_visualizer.visualize_information_flow(target_gene="RORB",select_topk=5,use_neuron_layer=True,cutoff=0.05)

# Cell subtyping analysis

We first construct interpretable CCI-informed features for each cell, (how each cell type influence each measured genes in this cell), use these features to do Leiden clustering and UMAP visualization. Then differential expressed gene (DEG) analysis can be performed on these subtypes (subgroups), and we can also visualize how this cell ('s one target gene) is influenced by other cell types via heatmap.

:param `sample`: which tissue slide to analyze

---

**You can tune the following hyperparameter to get a good subtyping results that you may want to see, different methods vary a lot**

:param `normalize_to_1`: whether normalize the aggregated influence tensor so that their abs value sum
        up to one on the second dimension
        
:param `use_abs`: whehter to use the absolute value for the aggregated influence tensor for downstream
        analysis
        
:param `noise_threshold`: For values in the influence tensor, if its corresponding proportional influence is less than this threshold,
        we would treat it as noise, ignore it by setting it to 0

In [ ]:
subtyping_analyzer=gitiii.subtyping_analyzer.Subtyping_anlayzer(sample=sample,normalize_to_1=True,use_abs=True,noise_threshold=2e-2)

In [ ]:
# Take L2/3 IT as an example for CCI informed subtyping analysis
COI="L2/3 IT" # Cell Of Interest
subtyping_analyzer.subtyping(COI=COI,resolution=0.1)

## Differential expressed gene analysis for the subgroups

function: subtyping_filter_groups(group_to_remain):

:param `group_to_remain`: list of str, for example, if you want to just analyze or compare the
            0-the group and 1-th group, as shown on the UMAP in subtyping analysis, you can make
            group_to_remain=\["1","0"\], be aware that the items in the list are not int, they are str

function: subtyping_DE(method='wilcoxon',n_gene_show=5)

:param `method`: statistical method to make comparison using scanpy, default to 'wilcoxon' (rank-sum test), other available methods are: 'logreg', 't-test', 'wilcoxon', 't-test_overestim_var'

:param `n_gene_show`: how many DE gene to plot for one subgroup

In [ ]:
# We only want to compare the subgroup 0 and subgroup 1, so we do this
subtyping_analyzer.subtyping_filter_groups(["0","1"])

subtyping_analyzer.subtyping_DE()

## Analyze, proportionally, how each cell in each subgroup influenced by other cell types

In [ ]:
subtyping_analyzer.subtyping_get_aggregated_influence()

Or, if we just want to look at the proportional influence targeting one gene, like RORB,

and we also want to know if it is up-regulated or down-regulated by other cell types, we can:

In [ ]:
subtyping_analyzer.subtyping_get_aggregated_influence_target_gene(target_gene="RORB")
# Down-regulation results in values less than 0, up-regulation results in values larger than 0

# Network analysis

Using partial linear regression to make statistical test of whether one cell type significantly influence one gene in the receiver cell type, forming a significant CCI network targeting each gene.

:param `noise_threshold`: For values in the influence tensor, if its corresponding proportional influence is less than this threshold,
        we would treat it as noise, ignore it by setting it to 0

In [ ]:
network_analyzer=gitiii.network_analyzer.Network_analyzer(noise_threshold=2e-2)

## Calculate the z-scores for the significant CCI network for one sample

In [ ]:
network_analyzer.determine_network_sample(sample=sample)

## Or if you want to calculate the significant CCI network for all samples

In [ ]:
network_analyzer.determine_networks()

## Calculate the counts for one sample and all samples

In [ ]:
# for one sample
network_analyzer.get_counts_sample(sample=sample)
# for all samples
network_analyzer.get_counts()